# From Excel to Python

In order to start converting the workflow out of Excel and into python, let's start with a small amount of data - easy enough to inspect visually - 10 rows, corresponding to a size of 10 GUs, and only 4 indicators for now. Values will be generated randomly

In [1]:
import pandas as pd
import numpy as np
import math
import random
import seaborn as sns

In [2]:
# get the data - will individual values already be in a table, or will we need to generate the table?
random.seed(10) # Just so results will be the same. However, the workflow is robust enough to work if the seed is changed!

d1 = {
    'GU': [i for i in range(1,11)],
    'Ind1': [random.uniform(0, 1) for i in range(10)],
    'Ind2': [random.uniform(0, 1) for i in range(10)],
    'Ind3': [random.uniform(0, 1) for i in range(10)],
    'Ind4': [random.uniform(0, 1) for i in range(10)]
}
df_indvals = pd.DataFrame(data=d1)
df_indvals.head()

,GU,Ind1,Ind2,Ind3,Ind4
0,1,0.571403,0.249997,0.685861,0.941002
1,2,0.428889,0.952817,0.661846,0.302861
2,3,0.578091,0.996557,0.132978,0.366146
3,4,0.206098,0.044556,0.767838,0.898196
4,5,0.813321,0.860161,0.982413,0.314364


In [3]:
df_indvals.set_index('GU', inplace=True)

In [4]:
df_indvals

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,0.571403,0.249997,0.685861,0.941002
2,0.428889,0.952817,0.661846,0.302861
3,0.578091,0.996557,0.132978,0.366146
4,0.206098,0.044556,0.767838,0.898196
5,0.813321,0.860161,0.982413,0.314364
6,0.823589,0.603191,0.969388,0.548982
7,0.653473,0.381606,0.613327,0.436031
8,0.160230,0.283618,0.044261,0.064994
9,0.520669,0.674965,0.004055,0.584546


In [5]:
df_rnk = df_indvals.rank(1,'first',ascending=False) #rank by column, not index; ties are ranked in order of first appearance
df_rnk.head()

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,3.0,4.0,2.0,1.0
2,3.0,1.0,2.0,4.0
3,2.0,1.0,4.0,3.0
4,3.0,4.0,2.0,1.0
5,3.0,2.0,1.0,4.0


In [6]:
ORness1 = 0.5 #App should allow for user to set ORness level, which in turn finds the correct set of weights
OWA1 = [0.25,0.25,0.25,0.25] #How are weights calculated? Is it better to save a list of weights or calculate them dynamically?

ORness2 = 0.95
OWA2 = [0.86892454,0.11411931,0.01498774,0.0019684]  #Correct weights should be mapped to position in matrix according to rank
#sort this list -> list becomes ranked by index

In [7]:
df_contrib = df_indvals * OWA1

In [8]:
df_contrib

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,0.142851,0.062499,0.171465,0.235251
2,0.107222,0.238204,0.165462,0.075715
3,0.144523,0.249139,0.033245,0.091536
4,0.051525,0.011139,0.191959,0.224549
5,0.203330,0.215040,0.245603,0.078591
6,0.205897,0.150798,0.242347,0.137246
7,0.163368,0.095401,0.153332,0.109008
8,0.040057,0.070905,0.011065,0.016249
9,0.130167,0.168741,0.001014,0.146137


In [9]:
df_contrib.sum(axis=1) #summed contributions are the OWA scores used for map

GU
1     0.612066
2     0.586603
3     0.518443
4     0.479172
5     0.742565
6     0.736287
7     0.521109
8     0.138276
9     0.446059
10    0.440661
dtype: float64

The above information yields us the OWA scores for ORness of 0.5. However, if the weights are not all equal, which holds true for all values of ORness > 0.5, an additional calculation is needed to calculate individual contributions. For each row, an individual value must be multiplied by the *correct* weight, determined by rank of magnitude.

In [10]:
#weights sorted in descending order, to be mapped according to individual value contribution rank
OWA2_ordered = sorted(OWA2,reverse=True)
OWA2_ordered

[0.86892454, 0.11411931, 0.01498774, 0.0019684]

In [11]:
df_rnkWt = df_rnk.applymap(lambda x: OWA2_ordered[int(x-1)]) # Maps the weight to each rank according to each weight's index in the list where the weights are found
df_rnkWt

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,0.014988,0.001968,0.114119,0.868925
2,0.014988,0.868925,0.114119,0.001968
3,0.114119,0.868925,0.001968,0.014988
4,0.014988,0.001968,0.114119,0.868925
5,0.014988,0.114119,0.868925,0.001968
6,0.114119,0.014988,0.868925,0.001968
7,0.868925,0.001968,0.114119,0.014988
8,0.114119,0.868925,0.001968,0.014988
9,0.014988,0.868925,0.001968,0.114119


Now we have a matrix comprised of rows of weights organized by rank based on the original dataframe of individual values, where the largest individual value receives the largest weight. As reference let's compare that with the original. However, can the dataframe function "applymap" continue to work efficiently for a much larger dataframe (1000 x 20?)? Does it remain feasible?

Now, each individual value from the original dataframe shall be multiplied by each corresponding weight for the individual contribution value (element-wise multiplication between the two dataframes).

In [12]:
df_indvals

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,0.571403,0.249997,0.685861,0.941002
2,0.428889,0.952817,0.661846,0.302861
3,0.578091,0.996557,0.132978,0.366146
4,0.206098,0.044556,0.767838,0.898196
5,0.813321,0.860161,0.982413,0.314364
6,0.823589,0.603191,0.969388,0.548982
7,0.653473,0.381606,0.613327,0.436031
8,0.160230,0.283618,0.044261,0.064994
9,0.520669,0.674965,0.004055,0.584546


In [13]:
df_contrib2 = df_indvals.mul(df_rnkWt)
df_contrib2

,Ind1,Ind2,Ind3,Ind4
GU,,,,
1,0.008564,0.000492,0.078270,0.817660
2,0.006428,0.827926,0.075529,0.000596
3,0.065971,0.865933,0.000262,0.005488
4,0.003089,0.000088,0.087625,0.780465
5,0.012190,0.098161,0.853643,0.000619
6,0.093987,0.009040,0.842325,0.001081
7,0.567818,0.000751,0.069992,0.006535
8,0.018285,0.246443,0.000087,0.000974
9,0.007804,0.586494,0.000008,0.066708


In [14]:
# Finding the OWA scores used for this ORNess of 0.95
df_contrib2.sum(axis=1)

GU
1     0.904986
2     0.910480
3     0.937654
4     0.871267
5     0.964613
6     0.946434
7     0.645097
8     0.265789
9     0.661013
10    0.790741
dtype: float64

## Using Real Data

Now that the workflow is more or less established, we can apply the same process to real data provided to compare and verify results. The given data actually had spaces for up to 1400 GU's and up to 20 indicators, but only 916 GU's were populated, and only 10 indicators were used. If they were kept in the below Excel file, then extra steps would be taken to take out all the null valued rows and columns.

In [15]:
df1 = pd.read_excel('C:/Users/mke/Documents/Current-Work/CBRN/Book1_MasterData.xlsx', index_col=0)
df1

,Ind 1,Ind2,Ind3,Ind4,Ind5,Ind6,Ind7,Ind8,Ind9,Ind10
GU,,,,,,,,,,
1,0.675215,0.375315,0.003536,0.820509,0.003191,0.292250,0.000000,0.000000,0.728324,0.239
2,0.213054,0.217884,0.000000,0.886046,0.001391,0.304764,0.000000,0.090909,0.508664,0.177
3,0.391738,0.295970,0.000000,0.892620,0.002047,0.305584,0.000000,0.000000,0.242863,0.125
4,0.808493,0.254408,0.000000,0.888119,0.000725,0.307093,0.000000,0.181818,0.666842,0.176
5,0.636851,0.304786,0.000000,0.867651,0.000756,0.299992,0.000000,0.090909,0.691851,0.197
...,...,...,...,...,...,...,...,...,...,...
912,0.729451,0.017632,0.121808,0.055035,0.373557,0.377380,1.000000,0.090909,0.822748,0.054
913,0.375236,0.030227,0.123061,0.185905,0.029139,0.090729,0.099270,0.090909,0.616689,0.193
914,0.430103,0.071788,0.089819,0.043605,0.292245,0.759849,0.996461,0.363636,0.917387,0.052


In [16]:
df1_rnk = df1.rank(1,'first',ascending=False)
df1_rnk.head()

,Ind 1,Ind2,Ind3,Ind4,Ind5,Ind6,Ind7,Ind8,Ind9,Ind10
GU,,,,,,,,,,
1,3.0,4.0,7.0,1.0,8.0,5.0,9.0,10.0,2.0,6.0
2,5.0,4.0,9.0,1.0,8.0,3.0,10.0,7.0,2.0,6.0
3,2.0,4.0,8.0,1.0,7.0,3.0,9.0,10.0,5.0,6.0
4,2.0,5.0,9.0,1.0,8.0,4.0,10.0,6.0,3.0,7.0
5,3.0,4.0,9.0,1.0,8.0,5.0,10.0,7.0,2.0,6.0


In [17]:
# Once again, we'll have to copy / paste weight values for now. We'll use an ORNess = 0.6.
# Also, when copying from the tables, taking the step to order them in descending order is redundant.
OWA3 = [0.15690136,0.14036821,0.12557721,0.11234477,0.10050668,0.089916,0.08044129,0.07196496,0.06438181,0.05759771]

In [18]:
df1_rnkWt = df1_rnk.applymap(lambda x: OWA3[int(x-1)])
df1_rnkWt.head()

,Ind 1,Ind2,Ind3,Ind4,Ind5,Ind6,Ind7,Ind8,Ind9,Ind10
GU,,,,,,,,,,
1,0.125577,0.112345,0.080441,0.156901,0.071965,0.100507,0.064382,0.057598,0.140368,0.089916
2,0.100507,0.112345,0.064382,0.156901,0.071965,0.125577,0.057598,0.080441,0.140368,0.089916
3,0.140368,0.112345,0.071965,0.156901,0.080441,0.125577,0.064382,0.057598,0.100507,0.089916
4,0.140368,0.100507,0.064382,0.156901,0.071965,0.112345,0.057598,0.089916,0.125577,0.080441
5,0.125577,0.112345,0.064382,0.156901,0.071965,0.100507,0.057598,0.080441,0.140368,0.089916


In [19]:
df1_contrib = df1.mul(df1_rnkWt)
df1_contrib.head()

,Ind 1,Ind2,Ind3,Ind4,Ind5,Ind6,Ind7,Ind8,Ind9,Ind10
GU,,,,,,,,,,
1,0.084792,0.042165,0.000284,0.128739,0.000230,0.029373,0.0,0.000000,0.102233,0.021490
2,0.021413,0.024478,0.000000,0.139022,0.000100,0.038271,0.0,0.007313,0.071400,0.015915
3,0.054988,0.033251,0.000000,0.140053,0.000165,0.038374,0.0,0.000000,0.024409,0.011239
4,0.113487,0.025570,0.000000,0.139347,0.000052,0.034500,0.0,0.016348,0.083740,0.014158
5,0.079974,0.034241,0.000000,0.136136,0.000054,0.030151,0.0,0.007313,0.097114,0.017713


In [20]:
df1_contrib.sum(axis=1)

GU
1      0.409306
2      0.317913
3      0.302479
4      0.427202
5      0.402697
         ...   
912    0.470651
913    0.233313
914    0.509934
915    0.198835
916    0.451245
Length: 916, dtype: float64

It turns out that even for this sample data, the calculations still only take a split second - promising in terms of performance. The final issue to solve is how to figure out the weights. Can we calculate for the weights given a desired ORness level and the number of indicators?

## Calculating Weights?

Conceptually, while it may be feasible to store all the weights for all ORNess levels in their own tables and call for them when needed, the most elegant solution seems like calculating for the desired weights and using only those for a single use of the application. That way, we calculate and store only a 1xn (n=number of indicators) array of values instead of storing countless tables filled with values and then retrieving that same 1xn array from a given table.

However, calculating for the weights requires nonlinear optimization with constraints and may affect the live performance of the end-resulting application. To define the problem, (1) we first start by deciding how many weights are needed. The number of weights is equal to the number of indicators used. The first equation required is (2) the entropy equation that must be optimzed (see memo). This is subject to a constraint (3) based on the desired ORness, and the values for the weights are bounded by 0 and 1 (4).

Gekko may be a suitable package for this nonlinear optimization problem. It seems to be able to solve for multiple variables at once, and it has built-in functionality for looking to either maximize or minimize a value.

In [21]:
#pip install gekko - already installed
from gekko import GEKKO
m = GEKKO()

In [22]:
# initialize set of variables?
# We'll create an empty list, where each position represents the variable weight to be calculated
n = df1.shape[1] # df1 is 916 x 10, so n=10 for this data set
vl = [None] * n
vl

[None, None, None, None, None, None, None, None, None, None]

In [23]:
# syntax: x1 = m.Var(value=1,lb=1,ub=5)
vl[0] = m.Var(value=0.1, lb=0, ub=1)

# assigning all values in vl as gekko variables to be solved
for i in range(n):
    vl[i] = m.Var(value=1/n, lb=0, ub=1)

vl

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [24]:
# Use gekko array instead of python list??

x = m.Array(m.Var,(n))
# intial guess
ig = [1/n for i in range(n)]
# m.Var definition
i = 0
for xi in x:
    xi.value = ig[i]
    xi.lower = 0.00000001
    xi.upper = 1
    i += 1

In [25]:
# Writing out sums first
# expression "vl.index(wt) for wt in vl" doesn't work??
#sum((n-(1+i))*x[i] for i in range(n))
-sum(x[i]*m.log(x[i]) for i in range(n))

(-((((((((((0+((v12)*(log(v12))))+((v13)*(log(v13))))+((v14)*(log(v14))))+((v15)*(log(v15))))+((v16)*(log(v16))))+((v17)*(log(v17))))+((v18)*(log(v18))))+((v19)*(log(v19))))+((v20)*(log(v20))))+((v21)*(log(v21)))))

In [26]:
# m.Equation() <-- sum of weights == 1
# m.Equation() <-- (1/(n-1))*sum of [(n-rnk)*weight] == ORNess
# m.Maximize(obj) <-- this is the maximum entropy expression to be maximized

m.Equation(sum(x)==1)
m.Equation((1/(n-1))*sum((n-(1+i))*x[i] for i in range(n))==0.5)
m.Maximize(-sum(x[i]*m.log(x[i]) for i in range(n)))

In [27]:
m.solve()
print(x)

apm 173.48.130.207_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           21
   Intermediates:            0
   Connections  :            0
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:             21
 Number of total equations: -            2
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :             19
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [28]:
# Again with ORNess = 0.95 this time

y = m.Array(m.Var,(n))
# intial guess
ig = [1/n for i in range(n)]
# m.Var definition
i = 0
for xi in y:
    xi.value = ig[i]
    xi.lower = 0.00000001
    xi.upper = 1
    i += 1

m.Equation(sum(y)==1)
m.Equation((1/(n-1))*sum((n-(1+i))*y[i] for i in range(n))==0.95)
m.Maximize(-sum(y[i]*m.log(y[i]) for i in range(n)))

In [29]:
m.solve()
print(y)

apm 173.48.130.207_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           31
   Intermediates:            0
   Connections  :            0
   Equations    :            6
   Residuals    :            6
 
 Number of state variables:             31
 Number of total equations: -            4
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :             27
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [30]:
y[1]

[0.21404766065]

Gekko successfully found solutions for ORNess =0.5 and 0.95, and they match up with the spreadsheet, meaning our desired weights were calculated properly. For some reason, rerunning the m.solve blocks causes the code to give back errors - it could be something to do with gekko syntax, so let's keep the gekko blocks organized for now. Also, perhaps using intermediates to define the summation expressions could make the notation cleaner or perhaps optimize the solver's performance.